In [1]:
#Saturated Oxygen Data Correction

In [9]:
#This is to import libraries of different functions that allows the program do what it's doing
%pylab inline
from netCDF4 import Dataset
import pandas as pd
from pandas import Series , DataFrame
import csv
from IPython.core.display import Image
import urllib2
import StringIO
from thredds_crawler.crawl import Crawl

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = urllib2.urlopen("http://argo.jcommops.org/FTPRoot/Argo/Status/argo_all.txt").readlines()
data[1].replace('\0' , '').replace('\xff' , '').replace('\xfe' , '').replace('\r\n' , '')
#So this works for a specific row but not the whole thing O.o
#Make a loop that runs through all of the rows one by one (since all together doesn't work)

'3900380,27643,ARGOS," ",1510,09/02/2005,11/02/2005,02,2005,-56.47,-58.54," "," ",,0,0,APEX,"Turton, Jon",jon.turton@metoffice.gov.uk,Argo UK,UNITED KINGDOM,0,0,0,0,0,0,0,240,1500,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0'

In [8]:
d2 = [d.replace('\0' , '').replace('\xff' , '').replace('\xfe' , '').replace('\r\n' , '') for d in data]
d2[:5]

['WMO,TELECOM,TTYPE,MY_ID,SERIAL_NO,DATE0,NOTIF_DATE,MONTH0,YEAR0,LAT0,LON0,SHIP,CRUISE,DATE_,LAT_,LON_,MODEL,FULL_NAME,EMAIL,PROGRAM,COUNTRY,AGE,AGE_WEIGHT,GTS,PROFILES,DMPROFILES,GTSPRESS,GREYLIST,CYCLE_TIME,DRIFT_PRESS,PROFILE_PRESS,OXYGEN,FLUO,TURBIDITY,RAFOS,CDOM,BEAM,BACKSCATT,DOWNWELL,SST,SSS,PAL,OTHER,ICE_DETECT,ACTIVE,BEACHED,RETIRED',
 '3900380,27643,ARGOS," ",1510,09/02/2005,11/02/2005,02,2005,-56.47,-58.54," "," ",,0,0,APEX,"Turton, Jon",jon.turton@metoffice.gov.uk,Argo UK,UNITED KINGDOM,0,0,0,0,0,0,0,240,1500,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0',
 '3900381,27645,ARGOS," ",1511,09/02/2005,11/02/2005,02,2005,-58.78,-58.77," "," ",17/09/2009,-50.7143,33.9768,APEX,"Turton, Jon",jon.turton@metoffice.gov.uk,Argo UK,UNITED KINGDOM,1681.1643,0,1,168,168,2003,0,240,1500,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0',
 '3900382,27652,ARGOS," ",1512,09/02/2005,11/02/2005,02,2005,-57.49,-58.9," "," ",,0,0,APEX,"Turton, Jon",jon.turton@metoffice.gov.uk,Argo UK,UNITED KINGDOM,0,0,0,0,0,0,0,240,

In [5]:
#Latest attempt
df = pd.DataFrame(d2)
type(df)

pandas.core.frame.DataFrame

In [6]:
with open('d2.csv', 'w') as f:
    for row in d2:
        f.write(row + '\n')

df = pd.read_csv('d2.csv')
df

,WMO,TELECOM,TTYPE,MY_ID,SERIAL_NO,DATE0,NOTIF_DATE,MONTH0,YEAR0,LAT0,...,BACKSCATT,DOWNWELL,SST,SSS,PAL,OTHER,ICE_DETECT,ACTIVE,BEACHED,RETIRED
0,6900877,99731,ARGOS,5058,5058,29/06/2014,17/04/2014,6,2014,57.5161,...,0,0,0,0,0,0,0,1,0,0
1,6900876,99730,ARGOS,5057,5057,29/06/2014,17/04/2014,6,2014,56.9983,...,0,0,0,0,0,0,0,1,0,0
2,6902554,139249,ARGOS,OIN14DAR03,OIN14DAR03,23/06/2014,21/05/2014,6,2014,41.0021,...,0,0,0,0,0,0,0,1,0,0
3,6902553,139248,ARGOS,OIN14DAR02,OIN14DAR02,22/06/2014,21/05/2014,6,2014,38.9999,...,0,0,0,0,0,0,0,1,0,0
4,6902552,139247,ARGOS,OIN14DAR01,OIN14DAR01,21/06/2014,21/05/2014,6,2014,37.0001,...,0,0,0,0,0,0,0,1,0,0
5,6901848,300234060257590,IRIDIUM,,OIN14ITARI018,10/04/2014,28/03/2014,4,2014,34.2000,...,0,0,0,0,0,0,0,1,0,0
6,6901847,300234061528180,IRIDIUM,,OIN14ITARI017,10/04/2014,28/03/2014,4,2014,34.2000,...,0,0,0,0,0,0,0,1,0,0
7,6901846,300234061527160,IRIDIUM,,OIN14ITARI016,10/04/2014,28/03/2014,4,2014,35.3000,...,0,0,0,0,0,0,0,1,0,0
8,6901059,36038,ARGOS,172,172,25/07/2011,21/03/2011,7,2011,68.9984,...,0,0,0,0,0,0,0,0,0,0
9,6901058,35978,ARGOS,171,171,25/07/2011,21/03/2011,7,2011,69.0000,...,0,0,0,0,0,0,0,0,0,0


In [63]:
#This prints out certain columns (of my choosing) with all the rows still present
fd_oga = df.loc[: , ['WMO' , 'OXYGEN' , 'GREYLIST' , 'AGE' , 'DATE_' , 'LAT_' , 'LON_'] ] # oga = oxygen, grey list, and age
fd_oga

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
0,6900877,1,0,381.3470,16/07/2015,60.1843,-22.4563
1,6900876,1,0,381.1445,15/07/2015,59.3546,-11.6446
2,6902554,0,0,382.6879,10/07/2015,36.9802,-26.6489
3,6902553,0,0,392.3854,19/07/2015,35.7544,-27.7932
4,6902552,0,0,392.5143,18/07/2015,34.8368,-18.4200
5,6901848,0,0,443.9896,27/06/2015,34.7260,23.4200
6,6901847,0,0,446.9944,30/06/2015,37.5910,19.8750
7,6901846,0,0,444.9965,28/06/2015,36.1630,16.9910
8,6901059,0,0,1240.6757,17/12/2014,70.7842,7.8411
9,6901058,0,0,1230.5180,07/12/2014,65.8894,-1.4243


In [64]:
#This takes the fd_oga table and only shows the rows that have oxygen data
fd_oxygen = fd_oga.loc[ fd_oga.loc[: , 'OXYGEN'] == 1 , :]
fd_oxygen

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
0,6900877,1,0,381.3470,16/07/2015,60.1843,-22.4563
1,6900876,1,0,381.1445,15/07/2015,59.3546,-11.6446
40,1900943,1,0,317.3993,27/12/2007,20.6700,-31.6260
66,6902546,1,0,0.4931,21/01/2014,64.6896,0.0736
67,6902544,1,0,590.2862,29/06/2015,76.9700,-1.1120
69,6902545,1,0,521.7191,28/06/2015,69.3350,5.0190
70,6902547,1,0,526.0354,02/07/2015,68.2500,9.4520
82,6900799,1,0,1177.4587,20/09/2013,71.1470,5.3780
83,6900798,1,0,1825.5125,01/07/2015,78.5070,-2.2210
84,6900797,1,0,651.5580,13/03/2012,72.4208,8.8289


In [65]:
#This takes the fd_oga table and only shows the rows that aren't grey listed     #gl = grey list
fd_gl = fd_oga.loc[ fd_oga.loc[: , 'GREYLIST'] == 0 , :] #gl = grey list
fd_gl

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
0,6900877,1,0,381.3470,16/07/2015,60.1843,-22.4563
1,6900876,1,0,381.1445,15/07/2015,59.3546,-11.6446
2,6902554,0,0,382.6879,10/07/2015,36.9802,-26.6489
3,6902553,0,0,392.3854,19/07/2015,35.7544,-27.7932
4,6902552,0,0,392.5143,18/07/2015,34.8368,-18.4200
5,6901848,0,0,443.9896,27/06/2015,34.7260,23.4200
6,6901847,0,0,446.9944,30/06/2015,37.5910,19.8750
7,6901846,0,0,444.9965,28/06/2015,36.1630,16.9910
8,6901059,0,0,1240.6757,17/12/2014,70.7842,7.8411
9,6901058,0,0,1230.5180,07/12/2014,65.8894,-1.4243


In [66]:
#This takes the fd_oga table and only shows the rows that are at least 340 days old
fd_age = fd_oga.loc[ fd_oga.loc[: , 'AGE'] >= 340. , :]
fd_age

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
0,6900877,1,0,381.3470,16/07/2015,60.1843,-22.4563
1,6900876,1,0,381.1445,15/07/2015,59.3546,-11.6446
2,6902554,0,0,382.6879,10/07/2015,36.9802,-26.6489
3,6902553,0,0,392.3854,19/07/2015,35.7544,-27.7932
4,6902552,0,0,392.5143,18/07/2015,34.8368,-18.4200
5,6901848,0,0,443.9896,27/06/2015,34.7260,23.4200
6,6901847,0,0,446.9944,30/06/2015,37.5910,19.8750
7,6901846,0,0,444.9965,28/06/2015,36.1630,16.9910
8,6901059,0,0,1240.6757,17/12/2014,70.7842,7.8411
9,6901058,0,0,1230.5180,07/12/2014,65.8894,-1.4243


In [67]:
#Merging the fd_oxygen table with the fd_gl table
fd_merge_og = pd.merge(fd_oxygen , fd_gl) #og = oxygen and grey list
fd_merge_og

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
0,6900877,1,0,381.3470,16/07/2015,60.1843,-22.4563
1,6900876,1,0,381.1445,15/07/2015,59.3546,-11.6446
2,1900943,1,0,317.3993,27/12/2007,20.6700,-31.6260
3,6902546,1,0,0.4931,21/01/2014,64.6896,0.0736
4,6902544,1,0,590.2862,29/06/2015,76.9700,-1.1120
5,6902545,1,0,521.7191,28/06/2015,69.3350,5.0190
6,6902547,1,0,526.0354,02/07/2015,68.2500,9.4520
7,6900799,1,0,1177.4587,20/09/2013,71.1470,5.3780
8,6900798,1,0,1825.5125,01/07/2015,78.5070,-2.2210
9,6900797,1,0,651.5580,13/03/2012,72.4208,8.8289


In [68]:
#Later work will go off of this table

#FINALLY HAVE ONLY THE DATA I DESIRE
#PRAISE HELIX
#After all the trial and error (=^.^=)

fd_merge_oga = pd.merge(fd_merge_og , fd_age)
fd_merge_oga
#Merged the fd_merge_og table with the fd_age table

#Inactive -> active = 0

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
0,6900877,1,0,381.3470,16/07/2015,60.1843,-22.4563
1,6900876,1,0,381.1445,15/07/2015,59.3546,-11.6446
2,6902544,1,0,590.2862,29/06/2015,76.9700,-1.1120
3,6902545,1,0,521.7191,28/06/2015,69.3350,5.0190
4,6902547,1,0,526.0354,02/07/2015,68.2500,9.4520
5,6900799,1,0,1177.4587,20/09/2013,71.1470,5.3780
6,6900798,1,0,1825.5125,01/07/2015,78.5070,-2.2210
7,6900797,1,0,651.5580,13/03/2012,72.4208,8.8289
8,6900796,1,0,965.1496,19/01/2013,65.0640,-1.7090
9,4901043,1,0,816.5604,06/09/2012,26.8190,-86.0130


In [69]:
#May or may not even need this
gd = open('argo_global_meta.txt') #gd = global data
 
gd_table = pd.read_csv(gd , comment = '#')
gd_table
#Now to get it so when a user inputs a float number, the DAC of that number shows up

,file,profiler_type,institution,date_update
0,aoml/13857/13857_meta.nc,845,AO,20120521144513
1,aoml/13858/13858_meta.nc,845,AO,20130222100319
2,aoml/13859/13859_meta.nc,845,AO,20120521144513
3,aoml/15819/15819_meta.nc,845,AO,20120521144513
4,aoml/15820/15820_meta.nc,845,AO,20120521144513
5,aoml/15851/15851_meta.nc,845,AO,20130222100329
6,aoml/15852/15852_meta.nc,845,AO,20120521144512
7,aoml/15853/15853_meta.nc,845,AO,20120521144512
8,aoml/15854/15854_meta.nc,845,AO,20120521144512
9,aoml/15855/15855_meta.nc,845,AO,20120521144512


In [70]:
#Now to read it into pandas :DDD
#Thank you Based Mike ;)

In [71]:
#idea

# 20150708 H1553

#to have the program look for the specific DAC off of the Grey list file name
#Have the program unconcatenate it in a sense where you keep the first part (the DAC location) and delete the rest
#Then have the program associate that location with the specific float ID
#Hell yeah nig B)

In [4]:
a = 'abc'
b = a - 'bc'
b

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [5]:
a = 'abc'
b = a + 'bc'
b

'abcbc'

In [6]:
#So you can concatenate but not unconcatenate? Great....

In [21]:
c = Crawl("http://thredds.aodn.org.au/thredds/catalog/IMOS/Argo/dac/coriolis/6900877/profiles/catalog.xml")

In [22]:
c.datasets

[<LeafDataset id: IMOS/Argo/dac/coriolis/6900877/profiles/R6900877_075.nc, name: R6900877_075.nc, services: ['OPENDAP', 'HTTPServer', 'WMS']>,
 <LeafDataset id: IMOS/Argo/dac/coriolis/6900877/profiles/R6900877_074.nc, name: R6900877_074.nc, services: ['OPENDAP', 'HTTPServer', 'WMS']>,
 <LeafDataset id: IMOS/Argo/dac/coriolis/6900877/profiles/R6900877_073.nc, name: R6900877_073.nc, services: ['OPENDAP', 'HTTPServer', 'WMS']>,
 <LeafDataset id: IMOS/Argo/dac/coriolis/6900877/profiles/R6900877_072.nc, name: R6900877_072.nc, services: ['OPENDAP', 'HTTPServer', 'WMS']>,
 <LeafDataset id: IMOS/Argo/dac/coriolis/6900877/profiles/R6900877_071.nc, name: R6900877_071.nc, services: ['OPENDAP', 'HTTPServer', 'WMS']>,
 <LeafDataset id: IMOS/Argo/dac/coriolis/6900877/profiles/R6900877_070.nc, name: R6900877_070.nc, services: ['OPENDAP', 'HTTPServer', 'WMS']>,
 <LeafDataset id: IMOS/Argo/dac/coriolis/6900877/profiles/R6900877_069.nc, name: R6900877_069.nc, services: ['OPENDAP', 'HTTPServer', 'WMS']>,

In [52]:
from thredds_crawler.crawl import Crawl
c = Crawl("http://thredds.aodn.org.au/thredds/catalog/IMOS/Argo/dac/coriolis/1900650/profiles/catalog.xml")
urls = [s.get("url") for d in c.datasets for s in d.services if s.get("service").lower() == "opendap"]
print urls

['http://thredds.aodn.org.au/thredds/dodsC/IMOS/Argo/dac/coriolis/1900650/profiles/D1900650_137.nc', 'http://thredds.aodn.org.au/thredds/dodsC/IMOS/Argo/dac/coriolis/1900650/profiles/D1900650_135.nc', 'http://thredds.aodn.org.au/thredds/dodsC/IMOS/Argo/dac/coriolis/1900650/profiles/D1900650_134.nc', 'http://thredds.aodn.org.au/thredds/dodsC/IMOS/Argo/dac/coriolis/1900650/profiles/D1900650_133.nc', 'http://thredds.aodn.org.au/thredds/dodsC/IMOS/Argo/dac/coriolis/1900650/profiles/D1900650_132.nc', 'http://thredds.aodn.org.au/thredds/dodsC/IMOS/Argo/dac/coriolis/1900650/profiles/D1900650_131.nc', 'http://thredds.aodn.org.au/thredds/dodsC/IMOS/Argo/dac/coriolis/1900650/profiles/D1900650_130.nc', 'http://thredds.aodn.org.au/thredds/dodsC/IMOS/Argo/dac/coriolis/1900650/profiles/D1900650_129.nc', 'http://thredds.aodn.org.au/thredds/dodsC/IMOS/Argo/dac/coriolis/1900650/profiles/D1900650_128.nc', 'http://thredds.aodn.org.au/thredds/dodsC/IMOS/Argo/dac/coriolis/1900650/profiles/D1900650_127.nc',

In [53]:
abcdefg = Dataset("http://thredds.aodn.org.au/thredds/dodsC/IMOS/Argo/dac/coriolis/1900650/profiles/D1900650_137.nc")

In [54]:
for v in abcdefg.variables:
    print(v)

DATA_TYPE
FORMAT_VERSION
HANDBOOK_VERSION
REFERENCE_DATE_TIME
PLATFORM_NUMBER
PROJECT_NAME
PI_NAME
STATION_PARAMETERS
CYCLE_NUMBER
DIRECTION
DATA_CENTRE
DATE_CREATION
DATE_UPDATE
DC_REFERENCE
DATA_STATE_INDICATOR
DATA_MODE
INST_REFERENCE
WMO_INST_TYPE
JULD
JULD_QC
JULD_LOCATION
LATITUDE
LONGITUDE
POSITION_QC
POSITIONING_SYSTEM
PROFILE_PRES_QC
PROFILE_TEMP_QC
PROFILE_PSAL_QC
PROFILE_DOXY_QC
PROFILE_TEMP_DOXY_QC
PRES
PRES_QC
PRES_ADJUSTED
PRES_ADJUSTED_QC
PRES_ADJUSTED_ERROR
TEMP
TEMP_QC
TEMP_ADJUSTED
TEMP_ADJUSTED_QC
TEMP_ADJUSTED_ERROR
PSAL
PSAL_QC
PSAL_ADJUSTED
PSAL_ADJUSTED_QC
PSAL_ADJUSTED_ERROR
DOXY
DOXY_QC
DOXY_ADJUSTED
DOXY_ADJUSTED_QC
DOXY_ADJUSTED_ERROR
TEMP_DOXY
TEMP_DOXY_QC
TEMP_DOXY_ADJUSTED
TEMP_DOXY_ADJUSTED_QC
TEMP_DOXY_ADJUSTED_ERROR
PARAMETER
SCIENTIFIC_CALIB_EQUATION
SCIENTIFIC_CALIB_COEFFICIENT
SCIENTIFIC_CALIB_COMMENT
CALIBRATION_DATE
HISTORY_INSTITUTION
HISTORY_STEP
HISTORY_SOFTWARE
HISTORY_SOFTWARE_RELEASE
HISTORY_REFERENCE
HISTORY_DATE
HISTORY_ACTION
HISTORY_PARAM

In [58]:
abcdefg.variables['DOXY_ADJUSTED'][:]

array([[ 206.49000549,  206.38000488,  206.30000305,  206.8500061 ,
         206.86000061,  211.50999451,  218.30999756,  232.38000488,
         216.80000305,  190.08000183,  164.44999695,  126.55000305,
         104.45999908,   99.66000366,   97.80999756,   97.76000214,
          96.22000122,  101.37999725,  103.04000092,  102.47000122,
         100.73000336,   95.27999878,   96.        ,   98.54000092,
          96.81999969,   94.47000122,   82.45999908,   79.83999634,
          77.87999725,   71.45999908,   68.62999725,   67.12999725,
          64.98999786,   63.97000122,   63.99000168,   63.24000168,
          62.09999847,   59.84999847,   58.36000061,   57.38000107,
          57.84999847,   58.97000122,   57.09999847,   55.18000031,
          55.47999954,   57.13000107,   57.47000122,   58.54999924,
          57.99000168,   60.95000076,   61.16999817,   61.95999908,
          62.93999863,   66.37000275,   69.12000275,   72.5       ,
          79.83999634,   90.61000061,  104.83999

In [56]:
abcdefg.variables['LATITUDE'][:]

array([ 6.031])

In [59]:
np.average(abcdefg.variables['DOXY_ADJUSTED'][:])

113.06773

In [73]:
print fd_merge_oga.values

[['6900877' 1L 0L ..., '16/07/2015' 60.1843 -22.4563]
 ['6900876' 1L 0L ..., '15/07/2015' 59.3546 -11.6446]
 ['6902544' 1L 0L ..., '29/06/2015' 76.97 -1.112]
 ..., 
 ['3900274' 1L 0L ..., '29/04/2006' -8.016 -25.5937]
 ['4900497' 1L 0L ..., '26/02/2007' 34.7538 -63.69]
 ['4900637' 1L 0L ..., '21/08/2009' 45.956 -128.049]]
